In [1]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [2]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
file_path = 'Uchoa_et_al_2014/X-n101-k25.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [3]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n101-k25 100 206 EUC_2D
Depot coords: [365. 689.]
First customer coords: [146. 180.]
Distance(0,1): 554.1137067425782


In [4]:
# from constructors import clarke_wright_parallel
# from utils import build_distance_matrix, is_feasible
# from plot_utils import save_solution, plot_solution_from_file

# # Clarke–Wright
# cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
# print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
#       "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

# # 1) Save to .sol
# sol_path = save_solution(inst, cw.routes, cw.cost)  # -> solutions/X-n101-k25/X-n101-k25.sol

# # 2) Plot from .sol
# png_path = plot_solution_from_file(inst, sol_path)    # -> solutions/X-n101-k25/X-n101-k25.png
# print("Wrote:", sol_path, "and", png_path)

In [5]:
# from local_search import local_search
# # Local search
# # Use verbose=True for quick logging, or pass your own logger
# ls = local_search(
#     coords=inst.coords,
#     demand=inst.demand,
#     Q=inst.capacity,
#     routes=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     verbose=True,
#     time_limit_sec=60.0,   # stop early, keep best
#     improvement_eps=1e-4
# )

# print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")
# #write the LS solution
# sol_path_ls = save_solution(inst, ls.routes, ls.cost)
# png_path_ls = plot_solution_from_file(inst, sol_path_ls)
# print("Wrote:", sol_path_ls, "and", png_path_ls)

In [6]:
# from ils import run_ils, ILSConfig
# cfg = ILSConfig(
#     time_limit_sec=70.0,
#     ls_time_slice_sec=2.0,
#     max_passes_without_gain=2,
#     k_nearest=20,
#     T_init_factor=0.01,
#     T_cooling=0.98,
#     seed=0,
#     verbose=True,
# )
# ils_out = run_ils(
#     coords=inst.coords,
#     demand=inst.demand,
#     Q=inst.capacity,
#     routes_initial=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     round_euclidean=False,
#     cfg=cfg,
# )

# print(f"[ILS] best_cost={ils_out.cost:.2f} routes={len(ils_out.routes)} in {ils_out.elapsed_sec:.1f}s (iters={ils_out.iterations})")

# # 3) Check, save, plot
# ok = is_feasible(ils_out.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers)
# print("Feasible:", ok)

# sol_path = save_solution(inst, ils_out.routes, ils_out.cost)
# png_path = plot_solution_from_file(inst, sol_path)
# print("Wrote:", sol_path, "and", png_path)

In [7]:
# from tabu_search import tabu_search

# ts = tabu_search(
#     coords=inst.coords,
#     demand=inst.demand,
#     Q=inst.capacity,
#     routes_init=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     time_limit_sec=70.0,             # hard wall
#     max_iters=100000,
#     max_no_improve=500,              # patience
#     tabu_tenure=15,                  # or tabu_tenure_range=(12,20)
#     aspiration=True,
#     k_nearest=20,
#     max_intra_candidates_per_route=64,
#     max_inter_candidates_per_route=128,
#     verbose=True,
#     seed=42,
# )

# print(f"[TS] routes={len(ts.routes)} cost={ts.cost:.2f}")

# # Save & plot
# sol_path = save_solution(inst, ts.routes, ts.cost)
# png_path = plot_solution_from_file(inst, sol_path)
# print("Wrote:", sol_path, "and", png_path)

In [8]:
# from alns import alns

# alns_sol = alns(
#   coords=inst.coords, demand=inst.demand, Q=inst.capacity,
#   routes_init=[r[:] for r in cw.routes],
#   edge_weight_type=inst.edge_weight_type,
#   # intensity & acceptance
#   remove_fraction=(0.08, 0.18),
#   start_temperature=500.0, end_temperature=1.0, cooling_rate=0.996,
#   use_destroy=("shaw","worst","clustered_routes"),
#   use_repair=("regret3","regret2"),
#   reaction=0.3, segment_length=40,
#   use_local_search=True, ls_time_limit_sec=1.2, ls_max_passes_without_gain=2,
#   time_limit_sec=120, max_iters=300000,
#   verbose=True, log_every=10, seed=44,
# )


# print(f"[ALNS] routes={len(alns_sol.routes)} cost={alns_sol.cost:.2f}")
# sol_path = save_solution(inst, alns_sol.routes, alns_sol.cost)
# png_path = plot_solution_from_file(inst, sol_path)
# print("Wrote:", sol_path, "and", png_path)

In [9]:
# from lns import lns

# lns_sol = lns(
#     coords=inst.coords, demand=inst.demand, Q=inst.capacity,
#     routes_init=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     # knobs
#     remove_fraction=(0.12, 0.25),
#     use_sa=True, start_temperature=900.0, end_temperature=5.0, cooling_rate=0.996,
#     # intensification
#     use_local_search=True, ls_time_limit_sec=1.0, ls_max_passes_without_gain=1,
#     # budget/logging
#     time_limit_sec=90, max_iters=200000,
#     verbose=True, log_every=5, log_detail=True,
#     seed=7,
# )

# print(f"[LNS] routes={len(lns_sol.routes)} cost={lns_sol.cost:.2f}")
# sol_path = save_solution(inst, lns_sol.routes, lns_sol.cost)
# png_path = plot_solution_from_file(inst, sol_path)
# print("Wrote:", sol_path, "and", png_path)

In [10]:
# # main_autotune_example.py
# import logging
# from cvrp_parser import parse_cvrplib_uchoa_2014
# from constructors import clarke_wright_parallel
# from autotune import autotune_and_run_all

# logging.basicConfig(level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s", datefmt="%H:%M:%S")
# logger = logging.getLogger("AUTO")

# inst = parse_cvrplib_uchoa_2014(open("Uchoa_et_al_2014/X-n101-k25.vrp","r",encoding="utf-8").read())

# # Seed solution (CW or your favorite)
# cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
# logger.info(f"[CW] routes={len(cw.routes)} cost={cw.cost:.2f}")

# report = autotune_and_run_all(
#     coords=inst.coords,
#     demand=inst.demand,
#     Q=inst.capacity,
#     routes_seed=[r[:] for r in cw.routes],
#     edge_weight_type=inst.edge_weight_type,
#     round_euclidean=False,
#     time_budgets_sec={'ils':10.0,'tabu':10.0,'alns':20.0,'lns':20.0},
#     seeds={'ils':42,'tabu':42,'alns':42,'lns':42},
#     logger=logger,
#     verbose=False,
# )

# for name, res in report.items():
#     logger.info(f"[{name.upper()}] cost={res['cost']:.2f} routes={len(res['routes'])} tuned_cfg={res['cfg']}")


In [11]:
def time_budgets_for_n(n: int) -> dict:
    """
    Return per-algorithm time budgets (seconds) based on #customers n (0..3000).
    Keys match autotune_and_run_all: 'ls','ils','tabu','alns','lns'.
    """
    if n <= 130:
        return {'ls': 6.0,  'ils': 40.0,  'tabu': 40.0,  'alns': 90.0,  'lns': 90.0}
    elif n <= 300:
        return {'ls': 8.0, 'ils': 50.0,  'tabu': 50.0,  'alns': 120.0,  'lns': 120.0}
    elif n <= 700:
        return {'ls': 10.0, 'ils': 90.0,  'tabu': 90.0,  'alns': 120.0, 'lns': 120.0}
    elif n <= 1500:
        return {'ls': 12.0, 'ils': 100.0, 'tabu': 100.0, 'alns': 120.0, 'lns': 120.0}
    else:  # 1501..3000
        return {'ls': 15.0, 'ils': 120.0, 'tabu': 120.0, 'alns': 120.0, 'lns': 120.0}


In [12]:
from autotune import autotune_and_run_all
import logging
import os

logging.basicConfig(level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s", datefmt="%H:%M:%S")
logger = logging.getLogger("AUTO")

# with open("Uchoa_et_al_2014/X-n101-k25.vrp","r",encoding="utf-8") as f:
#     inst = parse_cvrplib_uchoa_2014(f.read())

# report = autotune_and_run_all(
#     inst=inst,
#     time_budgets_sec=time_budgets_for_n(inst.n_customers),
#     seeds={'ls':42, 'ils':42, 'tabu':42, 'alns':42, 'lns':42},
#     logger=logger, verbose=True,
# )

In [13]:
#ok now we want a for loop to run for all files in Uchoa_et_al_2014 directory
import os
import random
i=0
filenames = os.listdir("Uchoa_et_al_2014")
#shuffle the filenames randomly
filenames = random.sample(filenames, len(filenames))
for filename in filenames:
    if filename.endswith(".vrp"):
        #print the filename
        print(filename)
        filepath = os.path.join("Uchoa_et_al_2014", filename)
        with open(filepath, "r", encoding="utf-8") as f:
            inst = parse_cvrplib_uchoa_2014(f.read())
        
        report = autotune_and_run_all(
            inst=inst,
            time_budgets_sec=time_budgets_for_n(inst.n_customers),
            seeds={'ls':42, 'ils':42, 'tabu':42, 'alns':42, 'lns':42},
            logger=logger, verbose=True,
        )
        i+=1
        # if i==3:
        #     break

00:51:15 [INFO] [CW] routes=14 cost=29853.07 time=0.12s
00:51:15 [INFO] [CW] routes=14 cost=29853.07 time=0.06s


X-n237-k14.vrp


00:51:15 [INFO] LS start | routes=14 cost=29853.0667 | time_limit=0.6s | no_gain_passes=1 | impr_eps=1e-06
00:51:15 [INFO]   -> new BEST after 2-opt r=0 i=3 k=10 | best=29852.7239
00:51:15 [INFO] [2-opt] r=0 i=3 k=10 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=0 i=3 k=8 | best=29830.5028
00:51:15 [INFO] [2-opt] r=0 i=3 k=8 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=2 i=2 k=3 | best=29794.8480
00:51:15 [INFO] [2-opt] r=2 i=2 k=3 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=2 i=3 k=4 | best=29774.4510
00:51:15 [INFO] [2-opt] r=2 i=3 k=4 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=2 i=9 k=11 | best=29772.5192
00:51:15 [INFO] [2-opt] r=2 i=9 k=11 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=2 i=13 k=14 | best=29768.3947
00:51:15 [INFO] [2-opt] r=2 i=13 k=14 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=6 i=12 k=13 | best=29749.5799
00:51:15 [INFO] [2-opt] r=6 i=12 k=13 kept
00:51:15 [INFO]   -> new BEST after 2-opt r=7 i=9 k=10 | best=29735.1760
00:51:15 [INFO] [

X-n153-k22.vrp


00:56:09 [INFO] LS start | routes=25 cost=22642.2860 | time_limit=0.6s | no_gain_passes=1 | impr_eps=1e-06
00:56:09 [INFO]   -> new BEST after 2-opt r=13 i=6 k=7 | best=22620.9160
00:56:09 [INFO] [2-opt] r=13 i=6 k=7 kept
00:56:09 [INFO]   -> new BEST after 2-opt r=19 i=5 k=6 | best=22620.6466
00:56:09 [INFO] [2-opt] r=19 i=5 k=6 kept
00:56:09 [INFO]   -> new BEST after Reloc-inter A=12 i=6 -> B=24 j=1 | best=22620.4551
00:56:09 [INFO] [Reloc-inter] A=12 i=6 -> B=24 j=1 kept
00:56:09 [INFO]   -> new BEST after Reloc-inter A=2 i=0 -> B=12 j=0 | best=22620.2420
00:56:09 [INFO] [Reloc-inter] A=2 i=0 -> B=12 j=0 kept
00:56:09 [INFO]   -> new BEST after Reloc-inter A=20 i=6 -> B=15 j=4 | best=22615.3158
00:56:09 [INFO] [Reloc-inter] A=20 i=6 -> B=15 j=4 kept
00:56:09 [INFO]   -> new BEST after Reloc-inter A=15 i=0 -> B=20 j=8 | best=22604.6450
00:56:09 [INFO] [Reloc-inter] A=15 i=0 -> B=20 j=8 kept
00:56:09 [INFO]   -> new BEST after Reloc-inter A=20 i=6 -> B=15 j=3 | best=22604.4499
00:56:

...

09:51:27 [INFO] [CW] routes=181 cost=166589.15 time=1.26s
09:51:28 [INFO] [CW] routes=181 cost=166589.15 time=1.27s
09:51:29 [INFO] LS start | routes=181 cost=166589.1492 | time_limit=1.0s | no_gain_passes=2 | impr_eps=1e-06
09:51:29 [INFO]   -> new BEST after 2-opt r=13 i=1 k=2 | best=166559.3832
09:51:29 [INFO] [2-opt] r=13 i=1 k=2 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=33 i=1 k=2 | best=166555.2546
09:51:29 [INFO] [2-opt] r=33 i=1 k=2 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=57 i=1 k=2 | best=166555.2228
09:51:29 [INFO] [2-opt] r=57 i=1 k=2 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=57 i=1 k=3 | best=166555.0181
09:51:29 [INFO] [2-opt] r=57 i=1 k=3 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=57 i=2 k=3 | best=166554.6683
09:51:29 [INFO] [2-opt] r=57 i=2 k=3 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=73 i=2 k=3 | best=166548.8886
09:51:29 [INFO] [2-opt] r=73 i=2 k=3 kept
09:51:29 [INFO]   -> new BEST after 2-opt r=75 i=1 k=2 | best=166547.6951
09:51

X-n308-k13.vrp


09:59:10 [INFO] [CW] routes=13 cost=28592.99 time=0.12s
09:59:10 [INFO] LS start | routes=13 cost=28592.9885 | time_limit=1.0s | no_gain_passes=2 | impr_eps=1e-06
09:59:10 [INFO]   -> new BEST after 2-opt r=0 i=12 k=13 | best=28548.1375
09:59:10 [INFO] [2-opt] r=0 i=12 k=13 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=0 i=35 k=36 | best=28545.4364
09:59:10 [INFO] [2-opt] r=0 i=35 k=36 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=0 i=32 k=36 | best=28544.1611
09:59:10 [INFO] [2-opt] r=0 i=32 k=36 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=0 i=30 k=36 | best=28543.3991
09:59:10 [INFO] [2-opt] r=0 i=30 k=36 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=0 i=40 k=41 | best=28538.5461
09:59:10 [INFO] [2-opt] r=0 i=40 k=41 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=1 i=16 k=18 | best=28513.9504
09:59:10 [INFO] [2-opt] r=1 i=16 k=18 kept
09:59:10 [INFO]   -> new BEST after 2-opt r=2 i=3 k=4 | best=28496.2573
09:59:10 [INFO] [2-opt] r=2 i=3 k=4 kept
09:59:10 [INFO]   -> n

X-n242-k48.vrp


10:04:47 [INFO] LS start | routes=49 cost=85023.5730 | time_limit=0.6s | no_gain_passes=1 | impr_eps=1e-06
10:04:47 [INFO]   -> new BEST after 2-opt r=6 i=2 k=3 | best=85011.6583
10:04:47 [INFO] [2-opt] r=6 i=2 k=3 kept
10:04:47 [INFO]   -> new BEST after 2-opt r=21 i=2 k=3 | best=85007.6405
10:04:47 [INFO] [2-opt] r=21 i=2 k=3 kept
10:04:47 [INFO]   -> new BEST after 2-opt r=28 i=1 k=2 | best=85007.0753
10:04:47 [INFO] [2-opt] r=28 i=1 k=2 kept
10:04:47 [INFO]   -> new BEST after 2-opt r=33 i=1 k=2 | best=84996.1071
10:04:47 [INFO] [2-opt] r=33 i=1 k=2 kept
10:04:47 [INFO]   -> new BEST after Or-opt3 r=17 i=2 j=1 | best=84996.0619
10:04:47 [INFO] [Or-opt3] r=17 i=2 j=1 kept
10:04:47 [INFO]   -> new BEST after Or-opt3 r=18 i=0 j=4 | best=84992.1090
10:04:47 [INFO] [Or-opt3] r=18 i=0 j=4 kept
10:04:47 [INFO]   -> new BEST after Or-opt2 r=34 i=3 j=2 | best=84989.1807
10:04:47 [INFO] [Or-opt2] r=34 i=3 j=2 kept
10:04:47 [INFO]   -> new BEST after Or-opt2 r=38 i=2 j=1 | best=84988.9151
10: